# Script para que el usuario ponga su documento, genere las firmas o pueda validar.

### Obtener el path del archivo que elija el usuario

In [2]:
import  tkinter as tk
from tkinter import filedialog
root = tk.Tk()
root.withdraw()
file_path = 'D:/Documentos/pdf-test.pdf'
print(file_path)

D:/Documentos/pdf-test.pdf


### Función que obtiene el hash del documento

In [3]:
def hashPDF(file, BLOCK_SIZE):
    # hash=sha256()
    with open(file, 'rb') as f: # Open the file to read it's bytes
        fb = f.read(BLOCK_SIZE) # Read from the file. Take in the amount declared above
        M = sl.sha256(fb)
    return M

size = os.path.getsize(file_path) 

### Generación firma Schnorr

In [43]:
import create_keypair as ckp
import schnorr_lib as sl
import statistics

user = ckp.create_keypair(1)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file_path, size)

sig = sl.schnorr_sign(M, user[0]["privateKey"])
#print("PublicKey =", user[0]["publicKey"])
print("Signature =", sig.hex())

Signature = cdc8b991623c8c20a79e589cf8e218177b6606cf1b9f0d5c4988ecd38d66859412795bc51108bbcad61964b02f919630cd4b35a26fb45a1fa6a4430aec0c2e00


## Tiempos para firmar el Documento

In [44]:
tiempos_firma = []
for i in range(1,100):
    t0 = time.perf_counter()
    sig = sl.schnorr_sign(M, user[0]["privateKey"])
    t1 = time.perf_counter()
    total = t1-t0
    tiempos_firma.append(total)

In [45]:
from statistics import mean

print(statistics.pstdev(tiempos_firma))

print(mean(tiempos_firma))

0.048778490982085636
0.3071531878791354


### Esquema Musig

In [11]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))
users = ckp.create_keypair(n_keys)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file_path, size)

sig, X, priv= sl.schnorr_musig_sign(M, users)
print(len(sig))
print(len(X))

print("Aggregated key =",len(X.hex()))
print("Signature =",len(sig.hex()))

77
77
64
32
Aggregated key = 64
Signature = 128


In [6]:
s = 'd66630062412391bf8f581a35c0dcb8746372fff045ad152458012601bf3163f17e56ac29f027958b336a9df168118b59e5b06cd936af44e0f6e40aeffe29c30'
len(s)

128

In [17]:
p = '9a722933fa4788f92e21ba2e848a347d939bfb012ec81a004b70fb8fa064641c'
len(p)

64

### Tiempos para Generar el Esquema

In [48]:
tiempos_esquema = []
for i in range(1,100):
    t0 = time.perf_counter()
    sig, X= sl.schnorr_musig_sign(M, users)
    t1 = time.perf_counter()
    total_1 = t1-t0
    tiempos_esquema.append(total_1)

In [47]:
from statistics import mean

print(statistics.pstdev(tiempos_esquema))

print(mean(tiempos_esquema))

0.01445772375121526
0.24727328686776215


### Esquema Musig 2

In [15]:
import create_keypair as ckp
import schnorr_lib as sl
import os
# from hashlib import sha256

n_keys = int(input("Insert the number of keys to be generated:"))
# n_keys = 2
users = ckp.create_keypair(n_keys)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file_path, size)

sig, X = sl.schnorr_musig2_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

Aggregated key = 0c3db4a9cc99398f8b4d4e502fc63b718dc1d3cc3e6c6729b363b9433f1e5e34
Signature = 93a00a37929054819cf485662290b8e895c14ad24abb3a0ccc74d5a216c1548174b97900c22da530607f33b395d55e915edc65b6a87dd1d26d4f8d664bd3fd37


### Verificar Firma Schnorr

In [53]:
import schnorr_lib as sl

# M = input("Insert the message to verify:")
# M = sl.sha256(M.encode())
M = hashPDF(file_path, size)

pubkey = input("Insert the public key (or the aggregated key if MuSig was used):")
pubkey_bytes = bytes.fromhex(pubkey)

sig = input("Insert the generated sign:")
sig_bytes = bytes.fromhex(sig)

result = sl.schnorr_verify(M, pubkey_bytes, sig_bytes)

if result:
    print("The signature is VALID for this message and this public key")
else:
    print("The signature is NOT VALID for this message and this public key")

The signature is VALID for this message and this public key


In [51]:
tiempos_validacion = []

pubkey = 'ada347b0f7df08a9f452839fbeed8f46ed294ceb7105f61ae7b9abfd28e758b1'
sig = '32054b80f10e640f5caab6d73de3334dd039db26c0604a8e22534c4ed6b62c3704dc8a7397a5a26dcc323d095776e13fefe95574da5912a4956139721fb52414'


for i in range(1,100):
    t0 = time.perf_counter()
    pubkey_bytes = bytes.fromhex(pubkey)

    sig_bytes = bytes.fromhex(sig)

    result = sl.schnorr_verify(M, pubkey_bytes, sig_bytes)
    t1 = time.perf_counter()
    total_3 = t1-t0
    tiempos_validacion.append(total_3)

In [52]:
from statistics import mean

print(statistics.pstdev(tiempos_validacion))

print(mean(tiempos_validacion))

0.009349623458082153
0.1197464282805514


In [1]:
import json
with open('users.json', 'r') as f:
            data = json.load(f)

dict = data['users'][0]['privateKey']
print(dict)

010e8a417c4a0f13a603a4566bb23678a8a896f1b9fafb4ac6d35bf0de081b26
